In [1]:
import pandas as pd
import numpy as np
from numpy.random import randn

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_importance
from lightgbm import LGBMClassifier
import xgboost as xgb
from lightgbm import plot_importance
from lightgbm import plot_importance

from scipy.stats import chi2_contingency
import scipy.stats as stats 
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.api import qqplot, add_constant
from statsmodels.api import Logit
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.proportion import proportions_ztest

from subprocess import call
from IPython.display import Image
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from yellowbrick.cluster import SilhouetteVisualizer

import graphviz
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.feature_selection import chi2
from sklearn.inspection import permutation_importance

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import silhouette_samples, silhouette_score
from imblearn.over_sampling import SMOTE

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

matplotlib.rc('font', family = 'NanumBarunGothic')
matplotlib.rc('axes', unicode_minus = False)

In [2]:
df_merged = pd.read_csv("/home/piai/merged_with_2.csv", encoding = 'euc-kr')

In [3]:
df_merged.shape

(14760, 57)

In [4]:
df_merged['Ox_Chamber'] = df_merged['Ox_Chamber'].astype(str)
df_merged['Lot_Num'] = df_merged['Lot_Num'].astype(str)
df_merged['Wafer_Num'] = df_merged['Wafer_Num'].astype(str)

df_merged['photo_soft_Chamber'] = df_merged['photo_soft_Chamber'].astype(str)

df_merged['lithography_Chamber'] = df_merged['lithography_Chamber'].astype(str)
df_merged['Wavelength'] = df_merged['Wavelength'].astype(str)

df_merged['Etching_Chamber'] = df_merged['Etching_Chamber'].astype(str)

# Flux480s, Flux840s, RTA_Temp 이산형 확인 .astype('category')
df_merged['Chamber_Num'] = df_merged['Chamber_Num'].astype(str)

df_merged['wafer_defect'] = df_merged['wafer_defect'].astype('category')
df_merged['Line_CD_state'] = df_merged['Line_CD_state'].astype('category')
df_merged['thickness_state'] = df_merged['thickness_state'].astype('category')

In [5]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Ox_Chamber            14760 non-null  object  
 1   type                  14760 non-null  object  
 2   Temp_OXid             14760 non-null  float64 
 3   Vapor                 14760 non-null  object  
 4   ppm                   14760 non-null  float64 
 5   Pressure              14760 non-null  float64 
 6   Oxid_time             14760 non-null  int64   
 7   thickness             14760 non-null  float64 
 8   Lot_Num               14760 non-null  object  
 9   Wafer_Num             14760 non-null  object  
 10  photo_soft_Chamber    14760 non-null  object  
 11  resist_target         14760 non-null  float64 
 12  N2_HMDS               14760 non-null  float64 
 13  pressure_HMDS         14760 non-null  float64 
 14  temp_HMDS             14760 non-null  float64 
 15  te

In [6]:
# # 숫자형 피처만 필터링
# numerical_features = df_merged.select_dtypes(include=['int64', 'float64']).columns

# anova_results = {}
# for feature in numerical_features:
#     feature_values = df_merged[feature]
    
#     p_values = []
#     for other_feature in numerical_features:
#         if other_feature != feature:
#             other_values = df_merged[other_feature]
#             try:
#                 # ANOVA 수행
#                 f_stat, p_value = f_oneway(feature_values, other_values)
#                 p_values.append((min(feature, other_feature), max(feature, other_feature), p_value))
#             except Exception as e:
#                 print(f"Error processing feature {feature} with {other_feature}: {e}")
#                 p_values.append((min(feature, other_feature), max(feature, other_feature), None))
    
#     # 피처별 ANOVA 결과를 데이터프레임에 추가
#     anova_results[feature] = p_values

# # 유의하지 않은 결과만 필터링 (p-value > 0.05)
# non_significant_results = []
# for feature, results in anova_results.items():
#     for f1, f2, p_value in results:
#         if p_value is not None and p_value > 0.05:
#             non_significant_results.append((f1, f2, p_value))

# # 중복 제거 (정렬된 피처 쌍에 대한 중복 제거)
# non_significant_results = list(set(non_significant_results))

# # 결과를 p-value 기준으로 오름차순 정렬
# non_significant_results_sorted = sorted(non_significant_results, key=lambda x: x[2])

# # 결과 데이터프레임 생성 및 출력 - Non-significant Results (p-value > 0.05):
#   Feature 1          Feature 2   p-value
# 0    Target  Temp_implantation  0.255976
# non_significant_df = pd.DataFrame(non_significant_results_sorted, columns=['Feature 1', 'Feature 2', 'p-value'])
# print("\nANOVA - Non-significant Results (p-value > 0.05):")
# print(non_significant_df)

In [7]:
# # 숫자형 피처만 필터링
# numerical_features = df_merged.select_dtypes(include=['int64', 'float64']).columns

# t_test_results = {}
# for feature in numerical_features:
#     feature_values = df_merged[feature]
    
#     p_values = []
#     for other_feature in numerical_features:
#         if other_feature != feature:
#             other_values = df_merged[other_feature]
#             try:
#                 # t-검정 수행
#                 t_stat, p_value = ttest_ind(feature_values, other_values, nan_policy='omit')
#                 p_values.append((min(feature, other_feature), max(feature, other_feature), p_value))
#             except Exception as e:
#                 print(f"Error processing feature {feature} with {other_feature}: {e}")
#                 p_values.append((min(feature, other_feature), max(feature, other_feature), None))
    
#     # 피처별 t-검정 결과를 데이터프레임에 추가
#     t_test_results[feature] = p_values

# # 유의하지 않은 결과만 필터링 (p-value > 0.05)
# non_significant_results = []
# for feature, results in t_test_results.items():
#     for f1, f2, p_value in results:
#         if p_value is not None and p_value > 0.05:
#             non_significant_results.append((f1, f2, p_value))

# # 중복 제거 (정렬된 피처 쌍에 대한 중복 제거)
# non_significant_results = list(set(non_significant_results))

# # 결과를 p-value 기준으로 오름차순 정렬
# non_significant_results_sorted = sorted(non_significant_results, key=lambda x: x[2])

# # 결과 데이터프레임 생성 및 출력
# non_significant_df = pd.DataFrame(non_significant_results_sorted, columns=['Feature 1', 'Feature 2', 'p-value'])
# print("\nT-test - Non-significant Results (p-value > 0.05) sorted by p-value:")
# print(non_significant_df)

In [8]:
# def chi2_statistic_and_pvalue(confusion_matrix):
#     chi2_stat, p_value, _, _ = chi2_contingency(confusion_matrix, correction=False)
#     return chi2_stat, p_value

# # 범주형 피처만 필터링
# categorical_features = df_merged.select_dtypes(include=['object', 'category']).columns

# chi2_results = []
# for feature in categorical_features:
#     feature_values = df_merged[feature].dropna()
    
#     for other_feature in categorical_features:
#         if other_feature != feature:
#             other_values = df_merged[other_feature].dropna()
            
#             # 교차표 생성
#             contingency_table = pd.crosstab(feature_values, other_values)
            
#             # Ensure the table has more than one row and column
#             if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
#                 try:
#                     # 카이제곱 통계량과 p-value 계산
#                     chi2_stat, p_value = chi2_statistic_and_pvalue(contingency_table)
#                     chi2_results.append((feature, other_feature, chi2_stat, p_value))
#                 except Exception as e:
#                     print(f"Error processing feature {feature} with {other_feature}: {e}")
#                     chi2_results.append((feature, other_feature, None, None))
#             else:
#                 chi2_results.append((feature, other_feature, None, None))

# # 결과를 데이터프레임으로 변환
# chi2_df = pd.DataFrame(chi2_results, columns=['Feature 1', 'Feature 2', 'Chi-Square Statistic', 'p-value'])

# # Feature Pair를 기준으로 중복 제거
# chi2_df['Feature Pair'] = chi2_df.apply(lambda x: tuple(sorted([x['Feature 1'], x['Feature 2']])), axis=1)
# chi2_df = chi2_df.drop_duplicates(subset=['Feature Pair'])

# # p-value가 높은 결과만 필터링 (예: p-value > 0.05)
# chi2_df_filtered = chi2_df[chi2_df['p-value'] > 0.05]

# # p-value 기준으로 오름차순 정렬
# chi2_df_sorted = chi2_df_filtered.sort_values(by='p-value')

# # 결과 출력
# print("\nChi-Square Statistic and p-value - Results with high p-values (sorted by p-value):")
# print(chi2_df_sorted[['Feature 1', 'Feature 2', 'Chi-Square Statistic', 'p-value']])

In [9]:
df_merged.drop(columns=['ppm', 'Defective_Rate_chip', 'wafer_defect'], inplace=True)

In [10]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 54 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Ox_Chamber            14760 non-null  object  
 1   type                  14760 non-null  object  
 2   Temp_OXid             14760 non-null  float64 
 3   Vapor                 14760 non-null  object  
 4   Pressure              14760 non-null  float64 
 5   Oxid_time             14760 non-null  int64   
 6   thickness             14760 non-null  float64 
 7   Lot_Num               14760 non-null  object  
 8   Wafer_Num             14760 non-null  object  
 9   photo_soft_Chamber    14760 non-null  object  
 10  resist_target         14760 non-null  float64 
 11  N2_HMDS               14760 non-null  float64 
 12  pressure_HMDS         14760 non-null  float64 
 13  temp_HMDS             14760 non-null  float64 
 14  temp_HMDS_bake        14760 non-null  float64 
 15  ti

In [11]:
# 데이터프레임 정의
df_char = df_merged.select_dtypes(include="object")
df_numeric = df_merged.select_dtypes(exclude="object")

# 숫자형 데이터 표준화
scaler = StandardScaler()
np_numeric_scaled = scaler.fit_transform(df_numeric)
df_numeric_scaled = pd.DataFrame(np_numeric_scaled, columns=df_numeric.columns, index=df_numeric.index)

# 범주형 데이터 원-핫 인코딩
df_encoded = pd.get_dummies(df_char, drop_first=True)

# 최종 데이터프레임 생성
df_final = df_numeric_scaled.join(df_encoded)

In [12]:
# 타겟 변수와 설명 변수 정의
df_final['Target'] = df_merged['Target']  # 타겟 변수 추가

# 데이터 분할
df_train, df_test = train_test_split(df_final, test_size=0.3, random_state=1234)

print('Train data size : {}'.format(df_train.shape)) 
print('Test data size : {}'.format(df_test.shape))

# 타겟 변수와 설명 변수 정의
X_train = df_train.drop(columns='Target')
y_train = df_train['Target']
X_test = df_test.drop(columns='Target')
y_test = df_test['Target']

print('X_train data size : {}'.format(X_train.shape)) 
print('y_train data size : {}'.format(y_train.shape))
print('X_test data size : {}'.format(X_test.shape)) 
print('y_test data size : {}'.format(y_test.shape))

Train data size : (10332, 149)
Test data size : (4428, 149)
X_train data size : (10332, 148)
y_train data size : (10332,)
X_test data size : (4428, 148)
y_test data size : (4428,)


In [13]:
# 후진 제거법 구현
def backward_elimination(X, y, significance_level=0.05):
    variables = X.columns.tolist()
    removed_variables = []
    
    while True:
        X_with_const = sm.add_constant(X[variables])
        model = sm.OLS(y, X_with_const).fit()
        
        # p-value가 가장 높은 변수 찾기
        p_values = model.pvalues
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            # p-value가 가장 높은 변수 제거
            excluded_variable = p_values.idxmax()
            if excluded_variable == 'const':
                break
            variables.remove(excluded_variable)
            removed_variables.append(excluded_variable)
            print(f"제거된 변수: {excluded_variable} (p-value: {max_p_value:.4f})")
        else:
            break
    
    return variables

# 변수 선택
selected_variables = backward_elimination(X_train, y_train)

# 최종 모델 생성
X_train_final = sm.add_constant(X_train[selected_variables])
final_model = sm.OLS(y_train, X_train_final).fit()

# 회귀 분석 결과 출력
print("최종 모델 요약 (후진 제거법 적용 후):")
print(final_model.summary())

제거된 변수: Wafer_Num_24 (p-value: 0.9753)
제거된 변수: N2_HMDS (p-value: 0.9522)
제거된 변수: Month (p-value: 0.8997)
제거된 변수: Wafer_Num_4 (p-value: 0.8907)
제거된 변수: Wafer_Num_12 (p-value: 0.8630)
제거된 변수: Error_message_[['Random']] (p-value: 0.8068)
제거된 변수: Lot_Num_6 (p-value: 0.7808)
제거된 변수: Wafer_Num_27 (p-value: 0.7792)
제거된 변수: Wafer_Num_25 (p-value: 0.7999)
제거된 변수: Wafer_Num_3 (p-value: 0.7886)
제거된 변수: Wafer_Num_18 (p-value: 0.8172)
제거된 변수: time_HMDS_bake (p-value: 0.7575)
제거된 변수: time_softbake (p-value: 0.7528)
제거된 변수: Wafer_Num_2 (p-value: 0.6530)
제거된 변수: Flux90s (p-value: 0.6287)
제거된 변수: Wafer_Num_26 (p-value: 0.6349)
제거된 변수: Wafer_Num_6 (p-value: 0.6515)
제거된 변수: IsWeekend (p-value: 0.5719)
제거된 변수: Lot_Num_4 (p-value: 0.5471)
제거된 변수: Lot_Num_15 (p-value: 0.6234)
제거된 변수: lithography_Chamber_2 (p-value: 0.5159)
제거된 변수: Wafer_Num_54 (p-value: 0.4867)
제거된 변수: Wafer_Num_21 (p-value: 0.4749)
제거된 변수: Wafer_Num_22 (p-value: 0.3777)
제거된 변수: Wafer_Num_50 (p-value: 0.3602)
제거된 변수: path (p-value: 0.9659)


In [14]:
# # 릿지 회귀 모델 및 하이퍼파라미터 그리드 정의
# ridge_model = Ridge()
# param_grid_ridge = {
#     'alpha': [0.0001, 0.001, 0.01 ,0.1, 1.0, 10.0, 100.0]  # alpha는 정규화 강도
# }

# # 그리드 서치 설정# 타겟 변수와 설명 변수 정의
# df_final['Target'] = df_merged['Target']  # 타겟 변수 추가

# # 데이터 분할
# df_train, df_test = train_test_split(df_final, test_size=0.3, random_state=1234)

# print('Train data size : {}'.format(df_train.shape)) 
# print('Test data size : {}'.format(df_test.shape))

# # 타겟 변수와 설명 변수 정의
# X_train = df_train.drop(columns='Target')
# y_train = df_train['Target']

# X_test = df_test.drop(columns='Target')
# y_test = df_test['Target']
# grid_search_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid_ridge, scoring='neg_mean_squared_error', cv=5)
# grid_search_ridge.fit(X_train, y_train)

# # 최적의 하이퍼파라미터와 성능 평가
# print(f"최적의 alpha: {grid_search_ridge.best_params_['alpha']}")
# print(f"최적의 MSE: {-grid_search_ridge.best_score_:.4f}")

# # 최적의 모델을 사용하여 테스트 데이터 평가
# best_ridge_model = grid_search_ridge.best_estimator_
# y_pred_ridge = best_ridge_model.predict(X_test)
# mse_ridge = mean_squared_error(y_test, y_pred_ridge)
# print(f"테스트 데이터 MSE (릿지 회귀): {mse_ridge:.4f}")

In [15]:
# # 라쏘 회귀 모델 및 하이퍼파라미터 그리드 정의
# lasso_model = Lasso()
# param_grid_lasso = {
#     'alpha': [0.0001, 0.001, 0.01 ,0.1, 1.0, 10.0, 100.0]  # alpha는 정규화 강도
# }

# # 그리드 서치 설정
# grid_search_lasso = GridSearchCV(estimator=lasso_model, param_grid=param_grid_lasso, scoring='neg_mean_squared_error', cv=5)
# grid_search_lasso.fit(X_train, y_train)

# # 최적의 하이퍼파라미터와 성능 평가
# print(f"최적의 alpha: {grid_search_lasso.best_params_['alpha']}")
# print(f"최적의 MSE: {-grid_search_lasso.best_score_:.4f}")

# # 최적의 모델을 사용하여 테스트 데이터 평가
# best_lasso_model = grid_search_lasso.best_estimator_
# y_pred_lasso = best_lasso_model.predict(X_test)
# mse_lasso = mean_squared_error(y_test, y_pred_lasso)
# print(f"테스트 데이터 MSE (라쏘 회귀): {mse_lasso:.4f}")

In [16]:
# # 엘라스틱넷 회귀 모델 및 하이퍼파라미터 그리드 정의
# elastic_net_model = ElasticNet()
# param_grid_en = {
#     'alpha': [0.0001, 0.001, 0.01 ,0.1, 1.0, 10.0, 100.0],  # alpha는 정규화 강도
#     'l1_ratio': [0.1, 0.5, 0.9]  # l1_ratio는 L1과 L2의 비율
# }

# # 그리드 서치 설정
# grid_search_en = GridSearchCV(estimator=elastic_net_model, param_grid=param_grid_en, scoring='neg_mean_squared_error', cv=5)
# grid_search_en.fit(X_train, y_train)

# # 최적의 하이퍼파라미터와 성능 평가
# print(f"최적의 alpha: {grid_search_en.best_params_['alpha']}")
# print(f"최적의 l1_ratio: {grid_search_en.best_params_['l1_ratio']}")
# print(f"최적의 MSE: {-grid_search_en.best_score_:.4f}")

# # 최적의 모델을 사용하여 테스트 데이터 평가
# best_en_model = grid_search_en.best_estimator_
# y_pred_en = best_en_model.predict(X_test)
# mse_en = mean_squared_error(y_test, y_pred_en)
# print(f"테스트 데이터 MSE (엘라스틱넷 회귀): {mse_en:.4f}")

In [17]:
# 릿지 회귀 모델 생성 및 학습
ridge_model = Ridge(alpha=0.0001, random_state=1234)  # alpha는 정규화 강도를 조절하는 하이퍼파라미터입니다.
ridge_model.fit(X_train, y_train)

# 예측
y_pred = ridge_model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

# 모델의 계수 출력
coefficients = pd.Series(ridge_model.coef_, index=X_train.columns)
print("\nRidge 회귀 계수:")
print(coefficients.sort_values(ascending=False))

Mean Squared Error: 928.4820
R^2 Score: 0.7475

Ridge 회귀 계수:
Lot_Num_21                       45.202713
Lot_Num_20                       40.497986
Lot_Num_25                       36.426804
Lot_Num_23                       35.246716
Lot_Num_22                       29.607187
                                   ...    
Wafer_Num_53                    -13.169893
Wafer_Num_38                    -14.934110
Wafer_Num_46                    -18.611250
Error_message_[['Edge-Ring']]   -31.515829
Error_message_none              -89.872233
Length: 148, dtype: float64


In [18]:
# 라쏘 회귀 모델 생성 및 학습
lasso_model = Lasso(alpha=0.0001, random_state=1234)  # alpha는 정규화 강도를 조절하는 하이퍼파라미터입니다.
lasso_model.fit(X_train, y_train)

# 예측
y_pred_lasso = lasso_model.predict(X_test)

# 성능 평가
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"라쏘 회귀 - Mean Squared Error: {mse_lasso:.4f}")
print(f"라쏘 회귀 - R^2 Score: {r2_lasso:.4f}")

# 모델의 계수 출력
coefficients_lasso = pd.Series(lasso_model.coef_, index=X_train.columns)
print("\n라쏘 회귀 계수:")
print(coefficients_lasso.sort_values(ascending=False))

라쏘 회귀 - Mean Squared Error: 928.4734
라쏘 회귀 - R^2 Score: 0.7475

라쏘 회귀 계수:
Lot_Num_21                       45.122262
Lot_Num_20                       40.417431
Lot_Num_25                       36.340467
Lot_Num_23                       35.165081
Lot_Num_22                       29.523617
                                   ...    
Wafer_Num_37                    -14.395417
Wafer_Num_38                    -14.751329
Wafer_Num_46                    -18.432151
Error_message_[['Edge-Ring']]   -31.450057
Error_message_none              -89.893733
Length: 148, dtype: float64


In [19]:
# 엘라스틱넷 회귀 모델 생성 및 학습
elastic_net_model = ElasticNet(alpha=0.0001, l1_ratio=0.9, random_state=1234)  # alpha는 정규화 강도, l1_ratio는 L1과 L2의 비율
elastic_net_model.fit(X_train, y_train)

# 예측
y_pred_en = elastic_net_model.predict(X_test)

# 성능 평가
mse_en = mean_squared_error(y_test, y_pred_en)
r2_en = r2_score(y_test, y_pred_en)

print(f"엘라스틱넷 회귀 - Mean Squared Error: {mse_en:.4f}")
print(f"엘라스틱넷 회귀 - R^2 Score: {r2_en:.4f}")

# 모델의 계수 출력
coefficients_en = pd.Series(elastic_net_model.coef_, index=X_train.columns)
print("\n엘라스틱넷 회귀 계수:")
print(coefficients_en.sort_values(ascending=False))

엘라스틱넷 회귀 - Mean Squared Error: 928.5204
엘라스틱넷 회귀 - R^2 Score: 0.7475

엘라스틱넷 회귀 계수:
Lot_Num_21                       44.932231
Lot_Num_20                       40.235775
Lot_Num_25                       36.172222
Lot_Num_23                       34.985498
Lot_Num_22                       29.340111
                                   ...    
Wafer_Num_37                    -13.660501
Wafer_Num_38                    -14.614699
Wafer_Num_46                    -18.300033
Error_message_[['Edge-Ring']]   -31.133145
Error_message_none              -89.751460
Length: 148, dtype: float64


In [20]:
# param_grid = {
#         'max_depth': [i for i in range(1,7)],
#         'min_samples_split': [i*10 for i in range(3,8)],
#         'min_samples_leaf': [i*5 for i in range(2,5)]}

# dt_reg = DecisionTreeRegressor()
# grid_search = GridSearchCV(estimator=dt_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)
# best_dt_reg = grid_search.best_estimator_
# best_params = best_dt_reg.get_params()
# print("Best Hyperparameters:", best_params)

In [21]:
# param_grid = {
#         'n_estimators': [50, 100, 200],
#         'max_depth': [i for i in range(1,7)],
#         'min_samples_split': [i*10 for i in range(3,8)],
#         'min_samples_leaf': [i*5 for i in range(2,5)]}

# rf_reg = RandomForestRegressor(random_state=1234)
# grid_search = GridSearchCV(estimator=rf_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)
# best_rf_reg = grid_search.best_estimator_
# best_params = best_rf_reg.get_params()
# print("Best Hyperparameters:", best_params)

In [ ]:
param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [i for i in range(1,7)],
        'min_samples_split': [i*10 for i in range(3,8)],
        'min_samples_leaf': [i*5 for i in range(2,5)]}

gb_reg = GradientBoostingRegressor(random_state=1234)
grid_search = GridSearchCV(estimator=gb_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_gb_reg = grid_search.best_estimator_
best_params = best_gb_reg.get_params()
print("Best Hyperparameters:", best_params)

In [ ]:
param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [i for i in range(1,7)],
        'min_samples_split': [i*10 for i in range(3,8)],
        'min_samples_leaf': [i*5 for i in range(2,5)]}

xgb_model = xgb.XGBRegressor(random_state=1234)
grid_search = GridSearchCV(estimator=gb_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(df_x, df_y)
best_gb_reg = grid_search.best_estimator_
best_params = best_gb_reg.get_params()
print("Best Hyperparameters:", best_params)

In [ ]:
param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [i for i in range(1,7)],
        'min_samples_split': [i*10 for i in range(3,8)],
        'min_samples_leaf': [i*5 for i in range(2,5)]}

lgb_model = lgb.LGBMRegressor(random_state=1234)
grid_search = GridSearchCV(estimator=gb_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(df_x, df_y)
best_gb_reg = grid_search.best_estimator_
best_params = best_gb_reg.get_params()
print("Best Hyperparameters:", best_params)

In [ ]:
# 결정 트리 회귀 모델 생성 및 학습
decision_tree_model = DecisionTreeRegressor(ccp_alpha = 0.0, criterion = 'squared_error', max_depth = 6, max_features = None, \
                                            max_leaf_nodes = None, min_impurity_decrease = 0.0, min_samples_leaf = 10, \
                                            min_samples_split = 30, min_weight_fraction_leaf = 0.0, random_state = 1234, splitter = 'best')
decision_tree_model.fit(X_train, y_train)

# 테스트 데이터 평가
y_pred_dt = decision_tree_model.predict(X_test)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f"테스트 데이터 MSE (결정 트리 회귀): {mse_dt:.4f}")

In [ ]:
# 피처 중요도
feature_importances_tree = pd.Series(decision_tree_model.feature_importances_, index=X_train.columns)
print("결정 트리 피처 중요도:")
print(feature_importances_tree.sort_values(ascending=False))

In [ ]:
# 랜덤 포레스트 회귀 모델 생성 및 학습
random_forest_model = RandomForestRegressor(bootstrap = True, ccp_alpha = 0.0, criterion = 'squared_error', max_depth = 6, \
                                            max_features = 1.0, max_leaf_nodes = None, max_samples = None, min_impurity_decrease = 0.0,\
                                            min_samples_leaf = 10, min_samples_split = 30, min_weight_fraction_leaf = 0.0,\
                                            n_estimators = 200, n_jobs = None, oob_score = False, random_state = 1234, verbose = 0, \
                                            warm_start = False)
random_forest_model.fit(X_train, y_train)

# 테스트 데이터 평가
y_pred_rf = random_forest_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"테스트 데이터 MSE (랜덤 포레스트 회귀): {mse_rf:.4f}")

In [ ]:
# 피처 중요도
feature_importances_forest = pd.Series(random_forest_model.feature_importances_, index=X_train.columns)
print("랜덤 포레스트 피처 중요도:")
print(feature_importances_forest.sort_values(ascending=False))

In [ ]:
# 그래디언트 부스팅 회귀 모델 생성 및 학습
gradient_boosting_model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
gradient_boosting_model.fit(X_train, y_train)

# 테스트 데이터 평가
y_pred_gb = gradient_boosting_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
print(f"테스트 데이터 MSE (그래디언트 부스팅 회귀): {mse_gb:.4f}")

In [ ]:
# 피처 중요도
feature_importances_gboost = pd.Series(gradient_boosting_model.feature_importances_, index=X_train.columns)
print("그래디언트 부스팅 피처 중요도:")
print(feature_importances_gboost.sort_values(ascending=False))

In [ ]:
# XGBoost 회귀 모델 생성 및 학습
xgb_model = xgb.XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# 테스트 데이터 평가
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f"테스트 데이터 MSE (XGBoost 회귀): {mse_xgb:.4f}")

In [ ]:
# 피처 중요도
feature_importances_xgb = pd.Series(xgb_model.feature_importances_, index=X_train.columns)
print("XGBoost 피처 중요도:")
print(feature_importances_xgb.sort_values(ascending=False))

In [ ]:
# LightGBM 회귀 모델 생성 및 학습
lgb_model = lgb.LGBMRegressor(n_estimators=n_estimators, learning_rate=learning_rate, num_leaves=num_leaves)
lgb_model.fit(X_train, y_train)

# 테스트 데이터 평가
y_pred_lgb = lgb_model.predict(X_test)
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
print(f"테스트 데이터 MSE (LightGBM 회귀): {mse_lgb:.4f}")

In [ ]:
# 피처 중요도
feature_importances_lgb = pd.Series(lgb_model.feature_importances_, index=X_train.columns)
print("LightGBM 피처 중요도:")
print(feature_importances_lgb.sort_values(ascending=False))